# Pre-processing the data obtained from the Karnataka Govt. website

## Importing the necessary libraries


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Reading the data

In [20]:
bengaluru_df = pd.read_csv('krama_report_bengaluru.csv')

all_markets = ['bengaluru', 'doddaballapur', 'hubballi', 'mysuru']


## Describing the data

In [21]:
bengaluru_df.head()

,Market,Date,Variety,Grade,Arrivals,Unit,Min,Max,Modal,District
0,BENGALURU,07/02/2002,ONION,FAQ,14350,Quintal,260,360,310,Bengaluru Urban�
1,BENGALURU,07/02/2002,OTHER,FAQ,0,Quintal,150,240,195,Bengaluru Urban�
2,BENGALURU,27/03/2002,ONION,FAQ,10818,Quintal,170,280,225,Bengaluru Urban�
3,BENGALURU,28/03/2002,ONION,FAQ,8048,Quintal,190,300,245,Bengaluru Urban�
4,BENGALURU,30/03/2002,ONION,FAQ,16590,Quintal,180,280,230,Bengaluru Urban�


In [22]:
bengaluru_df.describe()

,Arrivals,Min,Max,Modal
count,16993.000000,16993.000000,16993.000000,16993.000000
mean,8638.172895,991.099865,1414.385982,1196.057377
std,11639.161578,868.656112,3985.689866,986.092362
min,0.000000,0.000000,0.000000,0.000000
25%,978.000000,460.000000,650.000000,570.000000
50%,5099.000000,800.000000,1000.000000,900.000000
75%,12133.000000,1200.000000,1800.000000,1500.000000
max,173949.000000,13000.000000,500415.000000,14358.000000


In [23]:
bengaluru_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16993 entries, 0 to 16992
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Market    16993 non-null  object
 1   Date      16993 non-null  object
 2   Variety   16993 non-null  object
 3   Grade     16993 non-null  object
 4   Arrivals  16993 non-null  int64 
 5   Unit      16993 non-null  object
 6   Min       16993 non-null  int64 
 7   Max       16993 non-null  int64 
 8   Modal     16993 non-null  int64 
 9   District  16993 non-null  object
dtypes: int64(4), object(6)
memory usage: 1.3+ MB


## Cleaning the data

### Null values

In [24]:
def missing_values(df):
    print("The number of missing values in each column are:", df.isnull().sum())
    print("The number of missing values in the dataframe is:", df.isnull().sum().sum())
    
    # Drop rows with missing values
    df.dropna(inplace=True)
    print("The number of missing values after dropping rows with missing values is:", df.isnull().sum().sum())

missing_values(bengaluru_df)

The number of missing values in each column are: Market      0
Date        0
Variety     0
Grade       0
Arrivals    0
Unit        0
Min         0
Max         0
Modal       0
District    0
dtype: int64
The number of missing values in the dataframe is: 0
The number of missing values after dropping rows with missing values is: 0


Since arrivals hold no significance, we can remove the column.

### Converting Date to three columns: Day, Month, Year
We can use the split function to split the Date into three columns: Day, Month, and Year. We can then drop the Date column.

In [25]:
# Converting Date to three columns: year, month, day
def convert_date(df):
    date_format = r"%d/%m/%Y"
    df['Date'] = pd.to_datetime(df['Date'], format=date_format)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df.drop('Date', axis=1, inplace=True)

convert_date(bengaluru_df)
bengaluru_df.head()
    

,Market,Variety,Grade,Arrivals,Unit,Min,Max,Modal,District,Year,Month,Day
0,BENGALURU,ONION,FAQ,14350,Quintal,260,360,310,Bengaluru Urban�,2002,2,7
1,BENGALURU,OTHER,FAQ,0,Quintal,150,240,195,Bengaluru Urban�,2002,2,7
2,BENGALURU,ONION,FAQ,10818,Quintal,170,280,225,Bengaluru Urban�,2002,3,27
3,BENGALURU,ONION,FAQ,8048,Quintal,190,300,245,Bengaluru Urban�,2002,3,28
4,BENGALURU,ONION,FAQ,16590,Quintal,180,280,230,Bengaluru Urban�,2002,3,30


### Checking for redundant columns

#### Arrivals 0 values

In [26]:
# Arrivals = 0
no_arrivals = bengaluru_df[bengaluru_df['Arrivals'] == 0]
no_arrivals = no_arrivals.shape[0]
print("The number of rows with no arrivals are:", no_arrivals)


The number of rows with no arrivals are: 3465


In [27]:
# We can check the number of unique values in each column
def unique_values(df):
    for column in df.columns:
        print(column, ":", df[column].nunique())

unique_values(bengaluru_df)

Market : 1
Variety : 6
Grade : 5
Arrivals : 9739
Unit : 1
Min : 189
Max : 275
Modal : 337
District : 1
Year : 23
Month : 12
Day : 31


We can observe that the Market, Unit, Arrivals and District columns are redundant. We can drop these columns.

In [28]:
def remove_redundant_columns(df):
    red_cols = ["Market", "Unit", "District", "Arrivals"]
    df.drop(red_cols, axis=1, inplace=True)

remove_redundant_columns(bengaluru_df)
bengaluru_df.head()

,Variety,Grade,Min,Max,Modal,Year,Month,Day
0,ONION,FAQ,260,360,310,2002,2,7
1,OTHER,FAQ,150,240,195,2002,2,7
2,ONION,FAQ,170,280,225,2002,3,27
3,ONION,FAQ,190,300,245,2002,3,28
4,ONION,FAQ,180,280,230,2002,3,30


In [29]:
bengaluru_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16993 entries, 0 to 16992
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Variety  16993 non-null  object
 1   Grade    16993 non-null  object
 2   Min      16993 non-null  int64 
 3   Max      16993 non-null  int64 
 4   Modal    16993 non-null  int64 
 5   Year     16993 non-null  int32 
 6   Month    16993 non-null  int32 
 7   Day      16993 non-null  int32 
dtypes: int32(3), int64(3), object(2)
memory usage: 863.1+ KB


## Outliers Detection and Abnormalities Detection

### Outliers Detection

In [30]:
# Using Z-score to remove outliers
def find_zscore(values):
    mean = np.mean(values)
    std = np.std(values)
    z_scores = pd.Series([(i-mean)/std for i in values])
    return np.abs(z_scores)

def remove_outliers(df):
    outlier_cols = ['Min', 'Max', 'Modal']
    for column in outlier_cols:
        z_scores = find_zscore(df[column])
        # Remove rows with z-score > 3
        df = df.iloc[np.where(z_scores < 3)]
    return df

print("The number of rows before removing outliers are:", bengaluru_df.shape[0])
bengaluru_df = remove_outliers(bengaluru_df)
print("The number of rows after removing outliers are:", bengaluru_df.shape[0])


The number of rows before removing outliers are: 16993
The number of rows after removing outliers are: 16326


### Abnormalities Detection

In [31]:
# Here abnormalities are mainly the data in which the logical expression min <= modal <= max is not satisfied.
def remove_abnormalities(df):
    df = df[df['Min'] <= df['Modal']]
    df = df[df['Modal'] <= df['Max']]
    return df

print("The number of rows before removing abnormalities are:", bengaluru_df.shape[0])
bengaluru_df = remove_abnormalities(bengaluru_df)
print("The number of rows after removing abnormalities are:", bengaluru_df.shape[0])

The number of rows before removing abnormalities are: 16326
The number of rows after removing abnormalities are: 16293


## Converting to a Standard format

### One-hot encoding the Variety and Grade columns

In [32]:
def one_hot_encode(df):
    categorical_cols = ["Variety", "Grade"]
    return pd.get_dummies(df, columns=categorical_cols, drop_first=False)

bengaluru_df = one_hot_encode(bengaluru_df)
bengaluru_df.head()

,Min,Max,Modal,Year,Month,Day,Variety_BANGALORE SMALL,Variety_BELLARY RED,Variety_LOCAL,Variety_ONION,Variety_OTHER,Variety_PUNA,Grade_AVERAGE,Grade_FAQ,Grade_LARGE,Grade_MEDIUM,Grade_SMALL
0,260,360,310,2002,2,7,False,False,False,True,False,False,False,True,False,False,False
1,150,240,195,2002,2,7,False,False,False,False,True,False,False,True,False,False,False
2,170,280,225,2002,3,27,False,False,False,True,False,False,False,True,False,False,False
3,190,300,245,2002,3,28,False,False,False,True,False,False,False,True,False,False,False
4,180,280,230,2002,3,30,False,False,False,True,False,False,False,True,False,False,False


### Converting to float

In [33]:
# Converting to float
bengaluru_df = bengaluru_df.astype(float)
bengaluru_df.head()

,Min,Max,Modal,Year,Month,Day,Variety_BANGALORE SMALL,Variety_BELLARY RED,Variety_LOCAL,Variety_ONION,Variety_OTHER,Variety_PUNA,Grade_AVERAGE,Grade_FAQ,Grade_LARGE,Grade_MEDIUM,Grade_SMALL
0,260.0,360.0,310.0,2002.0,2.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,150.0,240.0,195.0,2002.0,2.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,170.0,280.0,225.0,2002.0,3.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,190.0,300.0,245.0,2002.0,3.0,28.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,180.0,280.0,230.0,2002.0,3.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Standardizing the data

Since the column Arrivals ranges over a large scale, we can standardize the data.

In [34]:
# def standardize_arrivals(df):
#     from sklearn.preprocessing import StandardScaler
#     scaler = StandardScaler()
#     df[['Arrivals']] = scaler.fit_transform(df[['Arrivals']])

# standardize_arrivals(bengaluru_df)
# bengaluru_df.head()

## Complete preprocessing for all datasets

### Helper function

In [35]:
def preprocess(df):
    print("Preprocessing the dataframe")
    print()
    print("Handling missing values")
    missing_values(df)
    print()
    print("Converting Date to three columns: year, month, day")
    convert_date(df)
    print()
    print("Removing redundant columns")
    remove_redundant_columns(df)
    print()
    print("One hot encoding categorical columns")
    df = one_hot_encode(df)
    print()
    print("Converting to float")
    df = df.astype(float)
    print()
    print("Removing outliers")
    print("The number of rows before removing outliers are:", df.shape[0])
    df = remove_outliers(df)
    print("The number of rows after removing outliers are:", df.shape[0])
    print()
    print("Removing abnormalities")
    print("The number of rows before removing abnormalities are:", df.shape[0])
    df = remove_abnormalities(df)
    print("The number of rows after removing abnormalities are:", df.shape[0])
    print()
    # print("Standardizing Arrivals")
    # standardize_arrivals(df)
    # print()
    return df

bengaluru_df = pd.read_csv('krama_report_bengaluru.csv')
bengaluru_df = preprocess(bengaluru_df)
bengaluru_df.head()

Preprocessing the dataframe

Handling missing values
The number of missing values in each column are: Market      0
Date        0
Variety     0
Grade       0
Arrivals    0
Unit        0
Min         0
Max         0
Modal       0
District    0
dtype: int64
The number of missing values in the dataframe is: 0
The number of missing values after dropping rows with missing values is: 0

Converting Date to three columns: year, month, day

Removing redundant columns

One hot encoding categorical columns

Converting to float

Removing outliers
The number of rows before removing outliers are: 16993
The number of rows after removing outliers are: 16326

Removing abnormalities
The number of rows before removing abnormalities are: 16326
The number of rows after removing abnormalities are: 16293



,Min,Max,Modal,Year,Month,Day,Variety_BANGALORE SMALL,Variety_BELLARY RED,Variety_LOCAL,Variety_ONION,Variety_OTHER,Variety_PUNA,Grade_AVERAGE,Grade_FAQ,Grade_LARGE,Grade_MEDIUM,Grade_SMALL
0,260.0,360.0,310.0,2002.0,2.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,150.0,240.0,195.0,2002.0,2.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,170.0,280.0,225.0,2002.0,3.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,190.0,300.0,245.0,2002.0,3.0,28.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,180.0,280.0,230.0,2002.0,3.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Cleaning the data and saving it

In [36]:
for market in all_markets:
    df = pd.read_csv(f'krama_report_{market}.csv')
    df = preprocess(df)
    print(f"Saving preprocessed dataframe for {market}")
    df.to_csv(f'preprocessed_krama_report_{market}.csv', index=False)
    

Preprocessing the dataframe

Handling missing values
The number of missing values in each column are: Market      0
Date        0
Variety     0
Grade       0
Arrivals    0
Unit        0
Min         0
Max         0
Modal       0
District    0
dtype: int64
The number of missing values in the dataframe is: 0
The number of missing values after dropping rows with missing values is: 0

Converting Date to three columns: year, month, day

Removing redundant columns

One hot encoding categorical columns

Converting to float

Removing outliers
The number of rows before removing outliers are: 16993
The number of rows after removing outliers are: 16326

Removing abnormalities
The number of rows before removing abnormalities are: 16326
The number of rows after removing abnormalities are: 16293

Saving preprocessed dataframe for bengaluru
Preprocessing the dataframe

Handling missing values
The number of missing values in each column are: Market      0
Date        0
Variety     0
Grade       0
Arriv